In [ ]:
import os
import re
import json
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack, csr_matrix, issparse


PATH_PRODUCTS = "../data/raw/products_with_prices_ingredients_nutrition.csv"  # enriched file
PATH_AISLES = "../data/raw/aisles.csv"
PATH_DEPTS = "../data/raw/departments.csv"

PATH_EXISTING_BUCKETS = None  # optional
OUT_DIR = "../data/kmeans_artifacts"
os.makedirs(OUT_DIR, exist_ok=True)


SIZE_PAT = re.compile(
    r"(?:\b\d+(?:\.\d+)?\s?(?:oz|ounce|ounces|lb|lbs|pound|g|gram|grams|kg|ml|l|liter|liters|ct|count|pack|pk)\b)",
    re.IGNORECASE,
)


def clean_name(s):
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(SIZE_PAT, " ", s)  # remove size tokens
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\b\d+\b", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


def extract_value_per_100g(row):
    price = row.get("Price_USD", np.nan)
    name = str(row.get("product_name", ""))
    sizes = SIZE_PAT.findall(name)
    mass_ml = None
    for token in sizes:
        t = token.lower().replace("ounces", "oz").replace("ounce", "oz")
        m = re.findall(r"(\d+(?:\.\d+)?)\s?(oz|lb|g|kg|ml|l)", t)
        if m:
            val, unit = m[0]
            val = float(val)
            if unit == "oz":
                mass_ml = val * 28.3495
            elif unit == "lb":
                mass_ml = val * 453.592
            elif unit == "g":
                mass_ml = val
            elif unit == "kg":
                mass_ml = val * 1000.0
            elif unit == "ml":
                mass_ml = val
            elif unit == "l":
                mass_ml = val * 1000.0
            break
    if pd.notnull(price) and mass_ml and mass_ml > 0:
        return price / (mass_ml / 100.0)
    return price


def quick_make_buckets(df, name_col="product_name", price_col="Price_USD", price_tol=0.12):
    base = df.copy()
    base["norm_name"] = base[name_col].fillna("").str.lower()
    base["norm_name"] = base["norm_name"].str.replace(r"[^a-z0-9\s]", " ", regex=True)
    base["norm_name_nosize"] = (
        base["norm_name"].str.replace(SIZE_PAT, " ", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()
    )

    bucket_ids = np.empty(len(base), dtype=object)
    next_id = 1
    for nn, sub in base.groupby("norm_name_nosize"):
        if nn == "":
            for idx in sub.index:
                bucket_ids[idx] = f"B{next_id:07d}"
                next_id += 1
            continue
        prices = sub[price_col].astype(float).fillna(sub[price_col].astype(float).median())
        med = prices.median()
        band_mask = (prices >= (1 - price_tol) * med) & (prices <= (1 + price_tol) * med)
        band_ids = []
        for idx in sub.index:
            band_ids.append("A" if band_mask.loc[idx] else "B")
        for idx, b in zip(sub.index, band_ids):
            bucket_ids[idx] = f"B{next_id:07d}_{b}"
            next_id += 1

    return bucket_ids


prod = pd.read_csv(PATH_PRODUCTS)
aisl = pd.read_csv(PATH_AISLES)
dept = pd.read_csv(PATH_DEPTS)

prod.columns = [c.strip() for c in prod.columns]
aisl.columns = [c.strip() for c in aisl.columns]
dept.columns = [c.strip() for c in dept.columns]

df = prod.merge(aisl, on="aisle_id", how="left").merge(dept, on="department_id", how="left")


df["value_per_100g"] = df.apply(lambda r: extract_value_per_100g(r), axis=1)


if "Price_USD" in df.columns:
    med_price = pd.to_numeric(df["Price_USD"], errors="coerce").median()
    df["Price_USD"] = pd.to_numeric(df["Price_USD"], errors="coerce").fillna(med_price)
else:
    df["Price_USD"] = pd.to_numeric(df["value_per_100g"], errors="coerce").fillna(
        pd.to_numeric(df["value_per_100g"], errors="coerce").median()
    )

if PATH_EXISTING_BUCKETS and os.path.exists(PATH_EXISTING_BUCKETS):
    p1 = pd.read_csv(PATH_EXISTING_BUCKETS)
    df = df.merge(p1[["product_id", "p1_bucket_id"]], on="product_id", how="left")
else:
    df["p1_bucket_id"] = quick_make_buckets(df)


df["name_clean"] = df["product_name"].map(clean_name).fillna("")
text_col = "name_clean"

df["aisle"] = df["aisle"].fillna("unknown_aisle")
df["department"] = df["department"].fillna("unknown_department")


nan_counts = {c: int(df[c].isna().sum()) for c in df.columns if df[c].isna().sum() > 0}
if nan_counts:
    print("[WARN] Columns with NaNs before feature engineering:", nan_counts)

tfidf = TfidfVectorizer(min_df=5, max_df=0.7, ngram_range=(1, 2))
svd = TruncatedSVD(n_components=64, random_state=42)
text_pipe = make_pipeline(tfidf, svd)
X_text = text_pipe.fit_transform(df[text_col])  # TruncatedSVD returns dense array


if np.isnan(X_text).any():
    print("[WARN] NaNs in X_text detected; replacing with 0s.")
    X_text = np.nan_to_num(X_text, nan=0.0)


if not issparse(X_text):
    X_text = csr_matrix(X_text)


num_feats = df[["Price_USD", "value_per_100g"]].astype(float)
num_feats = num_feats.fillna(num_feats.median())  # column-wise median fallback
num_scaler = RobustScaler()
X_num = num_scaler.fit_transform(num_feats)  # dense
if np.isnan(X_num).any():
    print("[WARN] NaNs in X_num after scaling; replacing with 0s.")
    X_num = np.nan_to_num(X_num, nan=0.0)
X_num = csr_matrix(X_num)  # sparse


try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)
X_aisle = ohe.fit_transform(df[["aisle"]].astype(str))

if hasattr(X_aisle, "toarray"):
    if np.isnan(X_aisle.toarray()).any():
        print("[WARN] NaNs in X_aisle detected; converting to 0s.")
        X_aisle = csr_matrix(np.nan_to_num(X_aisle.toarray(), nan=0.0))


X = hstack([X_text, X_num, X_aisle]).tocsr()

if np.isnan(X.data).any():
    print("[WARN] NaNs found in stacked feature matrix; replacing with 0s.")
    X.data = np.nan_to_num(X.data, nan=0.0)

sample_check_n = min(5000, X.shape[0])
sample_block = X[:sample_check_n].toarray()
if np.isnan(sample_block).any():

    print("[ERROR] NaNs still present in sample of stacked matrix — reporting column sources.")

    def block_has_nans(mat, name):
        if issparse(mat):
            arr = mat[:sample_check_n].toarray()
        else:
            arr = np.asarray(mat)[:sample_check_n]
        return np.isnan(arr).any()

    print(" X_text NaN?", block_has_nans(X_text, "X_text"))
    print(" X_num NaN?", block_has_nans(X_num, "X_num"))
    print(" X_aisle NaN?", block_has_nans(X_aisle, "X_aisle"))
    raise ValueError("Feature matrix contains NaNs after preprocessing. Inspect inputs.")
print(f"[OK] Feature matrix built: shape={X.shape}, sample NaN check passed.")


tfidf_params = {
    "min_df": int(tfidf.min_df),
    "max_df": float(tfidf.max_df) if tfidf.max_df is not None else None,
    "ngram_range": list(tfidf.ngram_range),
    "vocab_size": int(len(tfidf.vocabulary_)) if getattr(tfidf, "vocabulary_", None) is not None else None,
}
with open(os.path.join(OUT_DIR, "kmeans_feature_meta.json"), "w") as f:
    json.dump(
        {
            "tfidf_params": tfidf_params,
            "svd_n_components": int(svd.n_components),
            "numeric_cols": ["Price_USD", "value_per_100g"],
            "ohe_aisle_ncats": int(X_aisle.shape[1]),
        },
        f,
        indent=2,
    )


def sweep_kmeans_metrics(X, k_list, random_state=42, sample_for_scores=30000, densify_max=8000):
    n = X.shape[0]
    if n > sample_for_scores:
        rng = np.random.default_rng(random_state)
        idx = rng.choice(n, size=sample_for_scores, replace=False)
        Xs = X[idx]
    else:
        Xs = X

    results = []
    for k in k_list:
        km = KMeans(n_clusters=k, init="k-means++", n_init=10, random_state=random_state)
        km.fit(Xs)
        labels = km.labels_
        inertia = float(km.inertia_)

        sil = silhouette_score(Xs, labels, metric="cosine")

        if Xs.shape[0] > densify_max:
            rng = np.random.default_rng(random_state)
            sub_idx = rng.choice(Xs.shape[0], size=densify_max, replace=False)
            X_for_metrics = Xs[sub_idx]
            labels_for_metrics = labels[sub_idx]
        else:
            X_for_metrics = Xs
            labels_for_metrics = labels

        X_dense = X_for_metrics.toarray() if hasattr(X_for_metrics, "toarray") else np.asarray(X_for_metrics)
        dbi = davies_bouldin_score(X_dense, labels_for_metrics)
        ch = calinski_harabasz_score(X_dense, labels_for_metrics)
        results.append({"k": int(k), "inertia": inertia, "silhouette": float(sil) , "dbi": float(dbi), "calinski_h": float(ch)})
        print(f"[k={k}] inertia={inertia:.2f} silhouette={sil:.4f} dbi={dbi:.4f} calinski={ch:.1f}")
    return pd.DataFrame(results)


k_list = [24, 32, 40, 56, 64, 80, 96, 112, 128 , 150]
metrics_df = sweep_kmeans_metrics(X, k_list)
metrics_df.to_csv(os.path.join(OUT_DIR, "kmeans_metrics_sweep.csv"), index=False)


In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax[0].set_title("Elbow: Inertia vs K")
ax[0].set_xlabel("K")
ax[0].set_ylabel("Inertia")

ax[1].plot(metrics_df["k"], metrics_df["silhouette"], marker="o", label="Silhouette (↑)")
ax[1].plot(metrics_df["k"], metrics_df["dbi"], marker="s", label="Davies–Bouldin (↓)")
ax[1].plot(metrics_df["k"], metrics_df["calinski_h"], marker="^", label="Calinski–Harabasz (↑)")
ax[1].legend()
ax[1].set_title("Internal Metrics vs K")
ax[1].set_xlabel("K")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "kmeans_elbow_internal_metrics.png"), dpi=180)
plt.close()


BEST_K = int(metrics_df.sort_values(["silhouette", "calinski_h"], ascending=[False, False]).iloc[0]["k"])
print(f"Chosen K (heuristic): {BEST_K}")

kmeans = KMeans(n_clusters=BEST_K, init="k-means++", n_init=10, random_state=42)
labels_full = kmeans.fit_predict(X)
df["p2_cluster_id"] = labels_full
df.to_csv(os.path.join(OUT_DIR, "products_with_kmeans_labels.csv"), index=False)


def build_kpartite_graph(df, X, max_p1_per_cluster=30):
    G = nx.Graph()
    aisles = df["aisle"].fillna("unknown").unique().tolist()
    for a in aisles:
        G.add_node(("P3", a), layer="P3", label=a)

    clusters = sorted(df["p2_cluster_id"].unique().tolist())
    for c in clusters:
        G.add_node(("P2", int(c)), layer="P2", label=f"C{int(c)}")

    for (c, a), sub in df.groupby(["p2_cluster_id", "aisle"]):
        subs = sub.copy()
        if subs.shape[0] > max_p1_per_cluster:
            subs = subs.sample(max_p1_per_cluster, random_state=42)
        for _, row in subs.iterrows():
            p1 = row["p1_bucket_id"]
            if not G.has_node(("P1", p1)):
                G.add_node(("P1", p1), layer="P1", label=str(p1))
            G.add_edge(("P1", p1), ("P2", int(c)))
            G.add_edge(("P2", int(c)), ("P3", row["aisle"] if pd.notnull(row["aisle"]) else "unknown"))

    centroids = []
    cluster_list = clusters
    for c in cluster_list:
        idx = np.where(df["p2_cluster_id"].values == c)[0]
        if len(idx) > 2000:
            idx = np.random.default_rng(42).choice(idx, size=2000, replace=False)
        Xi = X[idx]
        cen = np.asarray(Xi.mean(axis=0)).reshape(-1)
        centroids.append(cen)

    C = np.vstack(centroids)
    nbrs = NearestNeighbors(n_neighbors=min(6, C.shape[0]), metric="cosine").fit(C)
    distances, indices = nbrs.kneighbors(C)
    for i, (drow, irow) in enumerate(zip(distances, indices)):
        c_i = cluster_list[i]
        for dist, j in zip(drow[1:], irow[1:]):
            c_j = cluster_list[j]
            sim = 1.0 - float(dist)
            if sim >= 0.75:
                G.add_edge(("P2", int(c_i)), ("P2", int(c_j)), weight=sim)

    return G


G = build_kpartite_graph(df, X)


def draw_kpartite(G, title="K-Partite Product Graph (KMeans)", out_path=None, sample_p1=800):
    p1_nodes = [n for n, d in G.nodes(data=True) if d.get("layer") == "P1"]
    if len(p1_nodes) > sample_p1:
        keep_idx = np.random.default_rng(1).choice(len(p1_nodes), size=sample_p1, replace=False)
        p1_keep = set([p1_nodes[i] for i in keep_idx])
        H = G.copy()
        for n in p1_nodes:
            if n not in p1_keep:
                H.remove_node(n)
    else:
        H = G

    layers = {"P1": 0, "P2": 1, "P3": 2}
    pos = {}
    for n, d in H.nodes(data=True):
        layer = d["layer"]
        pos[n] = (layers.get(layer, 1), np.random.random())

    color_map = {"P1": "#99d8c9", "P2": "#2ca25f", "P3": "#e5f5f9"}
    sizes = {"P1": 40, "P2": 250, "P3": 450}

    node_colors = [color_map[H.nodes[n]["layer"]] for n in H.nodes()]
    node_sizes = [sizes[H.nodes[n]["layer"]] for n in H.nodes()]

    plt.figure(figsize=(12, 8))
    nx.draw_networkx_edges(H, pos, alpha=0.25, width=0.6)
    nx.draw_networkx_nodes(H, pos, node_color=node_colors, node_size=node_sizes, linewidths=0.5, edgecolors="#444")
    labels = {n: d["label"] for n, d in H.nodes(data=True) if d["layer"] in ("P2", "P3")}
    nx.draw_networkx_labels(H, pos, labels=labels, font_size=8)

    import matplotlib.patches as mpatches

    patches = [
        mpatches.Patch(color=color_map["P1"], label="P1: Buckets"),
        mpatches.Patch(color=color_map["P2"], label="P2: KMeans Clusters"),
        mpatches.Patch(color=color_map["P3"], label="P3: Aisles"),
    ]
    plt.legend(handles=patches, loc="lower right")
    plt.title(title)
    plt.axis("off")
    if out_path:
        plt.savefig(out_path, dpi=200, bbox_inches="tight")
        plt.close()
    else:
        plt.show()


draw_kpartite(G, title=f"K-Partite Product Graph (KMeans, K={BEST_K})", out_path=os.path.join(OUT_DIR, "kpartite_kmeans.png"))


def draw_single_cluster(G, cluster_id, out_path=None):
    nodes_p2 = [("P2", int(cluster_id))]
    nbrs = set()
    for n in nodes_p2:
        nbrs.update(list(G.neighbors(n)))
    sub_nodes = set(nodes_p2) | nbrs
    H = G.subgraph(sub_nodes).copy()

    pos = nx.spring_layout(H, seed=42, k=0.45)
    color_map = {"P1": "#99d8c9", "P2": "#2ca25f", "P3": "#e5f5f9"}
    sizes = {"P1": 60, "P2": 400, "P3": 450}
    node_colors = [color_map[H.nodes[n]["layer"]] for n in H.nodes()]
    node_sizes = [sizes[H.nodes[n]["layer"]] for n in H.nodes()]

    plt.figure(figsize=(10, 7))
    nx.draw_networkx_edges(H, pos, alpha=0.3, width=0.8)
    nx.draw_networkx_nodes(H, pos, node_color=node_colors, node_size=node_sizes, edgecolors="#333", linewidths=0.6)
    labels = {n: H.nodes[n]["label"] for n in H.nodes() if H.nodes[n]["layer"] in ("P2", "P3")}
    nx.draw_networkx_labels(H, pos, labels=labels, font_size=9)
    plt.title(f"Cluster {cluster_id}: Neighborhood")
    plt.axis("off")
    if out_path:
        plt.savefig(out_path, dpi=200, bbox_inches="tight")
        plt.close()
    else:
        plt.show()


top_cluster = int(df["p2_cluster_id"].value_counts().index[0])
draw_single_cluster(G, top_cluster, out_path=os.path.join(OUT_DIR, f"cluster_{top_cluster}_neighborhood.png"))

print(f"[DONE] Artifacts at {OUT_DIR}")


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.collections import LineCollection

# --- Colors ---
C_P3 = '#e5f5f9'    # Aisle (P3)
C_P2 = '#2ca25f'    # KMeans cluster (P2)
C_BG = "#ffffff"

# Edges
C_EdgeP2P3 = '#696969'   # Dim gray
C_EdgeP2P2 = '#ff7f0e'   # Vibrant orange


def draw_p2p3_graph_enhanced_kmeans(G, title="Enhanced KMeans Substitution Graph"):

    # ---- CLEAN, EXTENDED LAYOUT ----
    pos = nx.spring_layout(
    G,
    seed=42,
    k=1.8,          # ⬅️ MUCH larger repulsion → BIGGER spacing
    iterations=120, # ⬅️ more stable spread
    weight='weight' # use similarity weights correctly
)

    # ---- Node groups ----
    p2_nodes = [n for n, d in G.nodes(data=True) if d.get("layer") == "P2"]
    p3_nodes = [n for n, d in G.nodes(data=True) if d.get("layer") == "P3"]

    # ---- Sizes ----
    p2_sizes = [700] * len(p2_nodes)
    p3_sizes = [320] * len(p3_nodes)

    # ---- Edge categories (corrected logic) ----
    e_p2p2 = [(u, v) for u, v, d in G.edges(data=True) if d.get("etype") == "P2-P2"]
    e_p2p3 = [(u, v) for u, v, d in G.edges(data=True) if d.get("etype") == "P2-P3"]

    p2p2_widths = [
        max(2, d.get("weight", 0.5) * 5)
        for u, v, d in G.edges(data=True) if d.get("etype") == "P2-P2"
    ]
    p2p3_widths = [2.8] * len(e_p2p3)

    # ---- Figure ----
    fig, ax = plt.subplots(figsize=(26, 20), dpi=160)
    ax.axis("off")
    ax.set_facecolor(C_BG)
    plt.title(title, fontsize=24, weight='bold')

    # Helper
    def segs(edge_list):
        return [(pos[u], pos[v]) for u, v in edge_list]

    # ---- Draw edges ----
    lc_p2p3 = LineCollection(
        segs(e_p2p3), colors=C_EdgeP2P3, linewidths=p2p3_widths, alpha=0.9
    )
    ax.add_collection(lc_p2p3)
    lc_p2p3.set_zorder(1)

    lc_p2p2 = LineCollection(
        segs(e_p2p2), colors=C_EdgeP2P2, linewidths=p2p2_widths, alpha=0.75
    )
    ax.add_collection(lc_p2p2)
    lc_p2p2.set_zorder(1.1)

    # ---- Draw nodes ----
    p3_coll = nx.draw_networkx_nodes(
        G, pos, nodelist=p3_nodes, node_color=C_P3, node_size=p3_sizes,
        edgecolors="#4a4a4a", linewidths=0.8, alpha=0.95, ax=ax
    )
    p3_coll.set_zorder(2)

    p2_coll = nx.draw_networkx_nodes(
        G, pos, nodelist=p2_nodes, node_color=C_P2, node_size=p2_sizes,
        edgecolors="#225e4d", linewidths=1.4, alpha=0.98, ax=ax
    )
    p2_coll.set_zorder(3)

    # ---- Labels ----
    p2_labels = {n: d["label"] for n, d in G.nodes(data=True) if d.get("layer") == "P2"}
    p2_texts = nx.draw_networkx_labels(
        G, pos, labels=p2_labels, font_size=13, font_weight='bold',
        font_color='#e5f5f9', ax=ax
    )
    for t in p2_texts.values(): t.set_zorder(4)

    p3_labels = {n: d["label"] for n, d in G.nodes(data=True) if d.get("layer") == "P3"}
    p3_texts = nx.draw_networkx_labels(
        G, pos, labels=p3_labels, font_size=11,
        font_color='#073642', ax=ax
    )
    for t in p3_texts.values(): t.set_zorder(3)


    legend_elems = [
        Line2D([0], [0], marker='o', color='w',
               label="P2: KMeans Cluster",
               markerfacecolor=C_P2, markeredgecolor='#225e4d', markersize=12),
        Line2D([0], [0], marker='o', color='w',
               label="P3: Aisle",
               markerfacecolor=C_P3, markeredgecolor='#4a4a4a', markersize=10),
        Line2D([0], [0], color=C_EdgeP2P2, lw=6,
               label="P2–P2 Similarity (Orange)"),
        Line2D([0], [0], color=C_EdgeP2P3, lw=3,
               label="P2–P3 Membership (Gray)"),
    ]
    ax.legend(handles=legend_elems, loc="upper left", fontsize=12, title="Graph Elements")

    plt.tight_layout()
    # plt.show()
    print("Enhanced P2–P3 visualization completed (decluttered).")



draw_p2p3_graph_enhanced_kmeans(G, title=f"KMeans Enhanced Substitution Graph (K={BEST_K})")


In [ ]:


from sklearn.decomposition import PCA


def compute_kmeans_bic_aic(X, k_list, metrics_df):

    n, d = X.shape
    bic_scores = []
    aic_scores = []

    for _, row in metrics_df.iterrows():
        k = int(row['k'])
        inertia = row['inertia']

        # Avoid log(0)
        log_likelihood = -n * np.log(max(inertia / n, 1e-10))

        bic = -2 * log_likelihood + k * np.log(n) * d
        aic = -2 * log_likelihood + 2 * k * d

        bic_scores.append(bic)
        aic_scores.append(aic)

    metrics_df['bic'] = bic_scores
    metrics_df['aic'] = aic_scores

    return metrics_df


metrics_df = compute_kmeans_bic_aic(X, k_list, metrics_df)


optimal_bic = metrics_df.loc[metrics_df['bic'].idxmin(), 'k']
optimal_silhouette = metrics_df.loc[metrics_df['silhouette'].idxmax(), 'k']
optimal_dbi = metrics_df.loc[metrics_df['dbi'].idxmin(), 'k']

print("\nOptimal K by criterion:")
print(f" BIC: {int(optimal_bic)}")
print(f" Silhouette: {int(optimal_silhouette)}")
print(f" Davies-Bouldin: {int(optimal_dbi)}")


fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)


ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(metrics_df['k'], metrics_df['bic'], 'o-', color='#1f77b4',
         linewidth=2, markersize=8, label='BIC (Lower is Better)')
ax1.plot(metrics_df['k'], metrics_df['aic'], 'o-', color='#ff7f0e',
         linewidth=2, markersize=8, label='AIC (Lower is Better)')


ax1.set_xlabel('K', fontsize=12, fontweight='bold')
ax1.set_ylabel('Score', fontsize=12, fontweight='bold')
ax1.set_title('KMeans: BIC/AIC Curve', fontsize=14, fontweight='bold', pad=15)
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3, linestyle='--')
ax1.tick_params(labelsize=10)


ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(metrics_df['k'], metrics_df['silhouette'], 'o-', color='#1f77b4',
         linewidth=2, markersize=8)


ax2.set_xlabel('K', fontsize=12, fontweight='bold')
ax2.set_ylabel('Score', fontsize=12, fontweight='bold')
ax2.set_title('KMeans: Silhouette Score', fontsize=14, fontweight='bold', pad=15)
ax2.grid(True, alpha=0.3, linestyle='--')
ax2.tick_params(labelsize=10)


ax3 = fig.add_subplot(gs[1, 0])
ax3.plot(metrics_df['k'], metrics_df['dbi'], 'o-', color='#1f77b4',
         linewidth=2, markersize=8)



ax3.set_xlabel('K', fontsize=12, fontweight='bold')
ax3.set_ylabel('Score', fontsize=12, fontweight='bold')
ax3.set_title('KMeans: Davies-Bouldin (Lower is Better)',
              fontsize=14, fontweight='bold', pad=15)
ax3.grid(True, alpha=0.3, linestyle='--')
ax3.tick_params(labelsize=10)


ax4 = fig.add_subplot(gs[1, 1])

max_samples_pca = 5000
if X.shape[0] > max_samples_pca:
    sample_idx = np.random.choice(X.shape[0], size=max_samples_pca, replace=False)
    X_pca_sample = X[sample_idx]
    labels_pca_sample = labels_full[sample_idx]
else:
    X_pca_sample = X
    labels_pca_sample = labels_full

X_dense_pca = X_pca_sample.toarray() if hasattr(X_pca_sample, 'toarray') else np.asarray(X_pca_sample)

pca = PCA(n_components=2, random_state=42)
X_pca_2d = pca.fit_transform(X_dense_pca)

colors = plt.cm.tab20(np.linspace(0, 1, len(np.unique(labels_pca_sample))))

for cluster_id in np.unique(labels_pca_sample):
    mask = labels_pca_sample == cluster_id
    ax4.scatter(
        X_pca_2d[mask, 0],
        X_pca_2d[mask, 1],
        c=[colors[cluster_id % len(colors)]],
        alpha=0.6,
        s=20,
        edgecolors='none',
        label=f"C{cluster_id}" if cluster_id < 10 else None
    )

ax4.set_xlabel('PC1', fontsize=12, fontweight='bold')
ax4.set_ylabel('PC2', fontsize=12, fontweight='bold')
ax4.set_title(f'KMeans PCA 2D View (K={BEST_K})', fontsize=14,
              fontweight='bold', pad=15)

if len(np.unique(labels_pca_sample)) <= 10:
    ax4.legend(loc='best', fontsize=8, ncol=2)

ax4.grid(True, alpha=0.3, linestyle='--')
ax4.tick_params(labelsize=10)

var_explained = pca.explained_variance_ratio_
ax4.text(
    0.02, 0.98,
    f"Explained var: PC1={var_explained[0]:.1%}, PC2={var_explained[1]:.1%}",
    transform=ax4.transAxes,
    fontsize=9,
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5)
)

plt.suptitle('KMeans Model Selection and Cluster Visualization',
             fontsize=16, fontweight='bold', y=0.995)

plt.savefig(os.path.join(OUT_DIR, 'kmeans_model_selection_dashboard.png'),
            dpi=200, bbox_inches='tight', facecolor='white')
plt.close()

print(f"[OK] Model selection dashboard saved to {OUT_DIR}/kmeans_model_selection_dashboard.png")

metrics_df.to_csv(os.path.join(OUT_DIR, "kmeans_metrics_sweep_full.csv"), index=False)



Optimal K by criterion:
 BIC: 24
 Silhouette: 112
 Davies-Bouldin: 150
[OK] Model selection dashboard saved to ../data/kmeans_artifacts/kmeans_model_selection_dashboard.png
